# Amazon Web Services

## S3

Creación de un bucket en S3

In [2]:
import boto3

In [ ]:
# Opción 1 - resource
s3r = boto3.resource('s3', region_name='us-east-1')
bucket = s3r.Bucket('s3castelar-python-resource')
bucket.create()


Cargar archivo json en el bucket creado anteriormente

In [ ]:
ficheroUpload = "datosPeliculas.json"
nombreBucket = "s3castelar-python-resource"


# Opción 2 - cliente
s3c = boto3.client('s3')

# 2.1 mediante upload_file
response = s3c.upload_file(ficheroUpload, nombreBucket, "datosSubidosC1.json")

Buckets de un usuario

In [2]:
print('Buckets mediante el cliente:')
s3client = boto3.client('s3')
response = s3client.list_buckets()
for bucket in response['Buckets']:
    print(f'\t{bucket["Name"]}')

Buckets mediante el cliente:
	s3castelar-python-resource


In [4]:
# Cliente: Ejemplo de como crear un objeto y añadirle contenido desde Python
nombreBucket = "s3castelar-python-resource"
s3client.put_object(Body=b'Ejemplo de datos binarios', Bucket=nombreBucket, Key="datosSubidosC3")

{'ResponseMetadata': {'RequestId': '4J0KBWF2F93G1QX3',
  'HostId': 'LIaX1PtP1Yup6NebMhAryXwJxkWQqTjrYw4CMJ9zfD7qlXKRjNxPjkqH/itR+DTxvffzQT7W23w=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'LIaX1PtP1Yup6NebMhAryXwJxkWQqTjrYw4CMJ9zfD7qlXKRjNxPjkqH/itR+DTxvffzQT7W23w=',
   'x-amz-request-id': '4J0KBWF2F93G1QX3',
   'date': 'Wed, 05 Apr 2023 09:52:54 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"212cbe310f4e00cb23aae95ba57d7907"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"212cbe310f4e00cb23aae95ba57d7907"',
 'ServerSideEncryption': 'AES256'}

Elementos de un bucket

In [5]:
s3 = boto3.resource('s3', region_name='us-east-1')
bucket = s3.Bucket(nombreBucket)
for obj in bucket.objects.all():
    print(obj.key)

datosSubidosC1.json
datosSubidosC3


## S3 Select

Subimos el archivo CSV donde después realizaremos la consulta

In [7]:
s3c = boto3.client('s3')

response = s3c.upload_file("TMDb_updated.csv", nombreBucket, "peliculas.csv")

Mediante S3Select ejecutaremos la consulta SELECT s.title, s.overview, s.vote_count, s.vote_average FROM s3object s WHERE cast(s.vote_count as int)> 10000 y almacenaremos el resultado en un nuevo CSV dentro del mismo bucket:

In [10]:
resp = s3c.select_object_content(
    Bucket=nombreBucket,
    Key='peliculas.csv',
    ExpressionType='SQL',
    Expression="SELECT s.title, s.overview, s.vote_count, s.vote_average FROM s3object s WHERE cast(s.vote_count as int)> 10000",
    InputSerialization={'CSV': {"FileHeaderInfo": "USE",
                                'AllowQuotedRecordDelimiter': True},
                        'CompressionType': 'NONE'},
    OutputSerialization={'CSV': {}},
)


# 2.- Unimos los datos que vamos recibiendo en streaming
registros = ["title,overview,vote_count,vote_average\n"]
for evento in resp['Payload']:
    if 'Records' in evento:
        registros.append(evento['Records']['Payload'].decode())

# 3.- Generamos el contenido en un String
file_str = ''.join(registros)

# 4.- Creamos un nuevo objeto en S3
s3c.put_object(Body=file_str, Bucket=nombreBucket,
              Key="TMDb_filtered.csv")

{'ResponseMetadata': {'RequestId': 'JWNRPJNM7HJM6N40',
  'HostId': 'UkeNqEggCF0WU7uAxYWwks0IB3s1LcVIluOwRCV90Ow0Uwr1vkU7al5o40LVFabS6cLUZ4v63cI=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'UkeNqEggCF0WU7uAxYWwks0IB3s1LcVIluOwRCV90Ow0Uwr1vkU7al5o40LVFabS6cLUZ4v63cI=',
   'x-amz-request-id': 'JWNRPJNM7HJM6N40',
   'date': 'Wed, 05 Apr 2023 10:19:48 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"0e46f2eef741ecda8647ca3b748b1432"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"0e46f2eef741ecda8647ca3b748b1432"',
 'ServerSideEncryption': 'AES256'}